In [1]:
'''
Objective:
To perform survival analysis on the provided ECG dataset, analyzing the survival time in hours after the operation and identifying factors influencing survival outcomes.
Constraints:
1.
Data Integrity: Ensure the accuracy and completeness of the ECG dataset, addressing any missing or erroneous data points before analysis.
2.
Ethical Considerations: Maintain patient confidentiality and adhere to ethical guidelines regarding the use of medical data for analysis.
3.
Statistical Rigor: Employ appropriate survival analysis techniques, considering censoring, time-to-event outcomes, and potential confounding variables, to derive meaningful insights.
4.
Interpretability: Ensure clear documentation and interpretation of analysis results, facilitating effective communication and decision-making.
Benefits/Impact:
1.
Medical Insights: The survival analysis provides valuable insights into the factors influencing patient survival after an operation, aiding medical professionals in understanding disease progression, treatment effectiveness, and prognosis.
2.
Treatment Optimization: By identifying factors associated with survival outcomes, healthcare providers can tailor treatment plans, allocate resources efficiently, and implement timely interventions to improve patient outcomes and quality of care.
3.
Resource Allocation: Insights from survival analysis enable healthcare facilities to allocate resources effectively, prioritize high-risk patients, and optimize healthcare delivery, leading to improved patient outcomes and cost-effectiveness.
4.
Research Advancement: Findings from survival analysis contribute to the body of medical knowledge, guiding future research endeavors and clinical trials aimed at developing better treatment protocols and interventions.

'''


import pandas as pd
from lifelines import CoxPHFitter

In [2]:
ecg = pd.read_excel('ECG_Surv.xlsx')
ecg

survival_time_hr  alive  age  pericardialeffusion  fractionalshortening  \
0                11.0      0   71                    0                 0.260   
1                19.0      0   72                    0                 0.380   
2                16.0      0   55                    0                 0.260   
3                57.0      0   60                    0                 0.253   
4                19.0      1   57                    0                 0.160   
..                ...    ...  ...                  ...                   ...   
128               7.5      1   64                    0                 0.240   
129              41.0      0   64                    0                 0.280   
130              36.0      0   69                    0                 0.200   
131              22.0      0   57                    0                 0.140   
132              20.0      0   62                    0                 0.150   

       epss   lvdd  wallmotion-score  wallmotion-index  multi_sensor  name  \
0     9.000  4.600              14.0             1.000         1.000  name   
1     6.000  4.100              14.0             1.700         0.588  name   
2     4.000  3.420              14.0             1.000         1.000  name   
3    12.062  4.603              16.0             1.450         0.788  name   
4    22.000  5.750              18.0             2.250         0.571  name   
..      ...    ...               ...               ...           ...   ...   
128  12.900  4.720              12.0             1.000         0.857  name   
129   5.400  5.470              11.0             1.100         0.714  name   
130   7.000  5.050              14.5             1.210         0.857  name   
131  16.100  4.360              15.0             1.360         0.786  name   
132   0.000  4.510              15.5             1.409         0.786  name   

     group  
0        1  
1        1  
2        1  
3        1  
4        1  
..     ...  
128      3  
129      3  
130      3  
131      3  
132      3  

[133 rows x 12 columns]

In [3]:
ecg.drop(['name'], axis = 1, inplace = True)


In [4]:
c = ecg.corr()

In [5]:
ecg.isna().sum()

survival_time_hr         0
alive                    0
age                      0
pericardialeffusion      0
fractionalshortening     0
epss                    16
lvdd                    12
wallmotion-score         5
wallmotion-index         3
multi_sensor             4
group                    0
dtype: int64

In [6]:
from sklearn.impute import SimpleImputer
impute = SimpleImputer(strategy= 'mean')
ecg1 = impute.fit_transform(ecg)
ecg_df = pd.DataFrame(ecg1, columns = ecg.columns)

In [7]:
ecg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   survival_time_hr      133 non-null    float64
 1   alive                 133 non-null    float64
 2   age                   133 non-null    float64
 3   pericardialeffusion   133 non-null    float64
 4   fractionalshortening  133 non-null    float64
 5   epss                  133 non-null    float64
 6   lvdd                  133 non-null    float64
 7   wallmotion-score      133 non-null    float64
 8   wallmotion-index      133 non-null    float64
 9   multi_sensor          133 non-null    float64
 10  group                 133 non-null    float64
dtypes: float64(11)
memory usage: 11.6 KB


In [8]:
ecg_df1 = ecg_df.astype('int')
ecg_df1

survival_time_hr  alive  age  pericardialeffusion  fractionalshortening  \
0                  11      0   71                    0                     0   
1                  19      0   72                    0                     0   
2                  16      0   55                    0                     0   
3                  57      0   60                    0                     0   
4                  19      1   57                    0                     0   
..                ...    ...  ...                  ...                   ...   
128                 7      1   64                    0                     0   
129                41      0   64                    0                     0   
130                36      0   69                    0                     0   
131                22      0   57                    0                     0   
132                20      0   62                    0                     0   

     epss  lvdd  wallmotion-score  wallmotion-index  multi_sensor  group  
0       9     4                14                 1             1      1  
1       6     4                14                 1             0      1  
2       4     3                14                 1             1      1  
3      12     4                16                 1             0      1  
4      22     5                18                 2             0      1  
..    ...   ...               ...               ...           ...    ...  
128    12     4                12                 1             0      3  
129     5     5                11                 1             0      3  
130     7     5                14                 1             0      3  
131    16     4                15                 1             0      3  
132     0     4                15                 1             0      3  

[133 rows x 11 columns]

In [9]:
#outlier analysis

In [10]:
ecg_df2 = ecg_df1.drop(['pericardialeffusion', 'fractionalshortening', 'wallmotion-index', 'multi_sensor', 'group'], axis = 1)
from feature_engine.outliers import Winsorizer
winsor = Winsorizer(capping_method = 'iqr',
                   tail = 'both',
                   fold = 1.5,
                   variables = list(ecg_df2.columns))
outlier_data = winsor.fit_transform(ecg_df2)
outlier_df = pd.DataFrame(outlier_data, columns = ecg_df2.columns)

In [11]:
cph = CoxPHFitter()
cph.fit(ecg_df2, duration_col='survival_time_hr', event_col='alive')

<lifelines.CoxPHFitter: fitted with 133 total observations, 82 right-censored observations>

In [12]:
cph.summary

coef  exp(coef)  se(coef)  coef lower 95%  \
covariate                                                         
age               0.031832   1.032344  0.017702       -0.002862   
epss              0.050104   1.051380  0.024107        0.002855   
lvdd             -0.074683   0.928038  0.215709       -0.497464   
wallmotion-score  0.070096   1.072611  0.027264        0.016660   

                  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%  \
covariate                                                                    
age                     0.066527             0.997142             1.068789   
epss                    0.097353             1.002859             1.102249   
lvdd                    0.348098             0.608071             1.416372   
wallmotion-score        0.123532             1.016800             1.131487   

                  cmp to         z         p  -log2(p)  
covariate                                               
age                  0.0  1.798265  0.072135  3.793155  
epss                 0.0  2.078379  0.037674  4.730271  
lvdd                 0.0 -0.346221  0.729177  0.455660  
wallmotion-score     0.0  2.571040  0.010139  6.623888

In [13]:
coefficients = cph.summary['coef']

In [14]:
hazard_ratio = cph.summary['exp(coef)']

In [15]:
coefficients

covariate
age                 0.031832
epss                0.050104
lvdd               -0.074683
wallmotion-score    0.070096
Name: coef, dtype: float64

In [16]:
hazard_ratio

covariate
age                 1.032344
epss                1.051380
lvdd                0.928038
wallmotion-score    1.072611
Name: exp(coef), dtype: float64

In [17]:
#propotional_hazard_ratio
propotional_test = cph.check_assumptions(training_df= ecg_df2)

Proportional hazard assumption looks okay.


In [18]:
propotional_test

[]

In [19]:
#propotional hazardtest is not violated

In [20]:
cph.summary

coef  exp(coef)  se(coef)  coef lower 95%  \
covariate                                                         
age               0.031832   1.032344  0.017702       -0.002862   
epss              0.050104   1.051380  0.024107        0.002855   
lvdd             -0.074683   0.928038  0.215709       -0.497464   
wallmotion-score  0.070096   1.072611  0.027264        0.016660   

                  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%  \
covariate                                                                    
age                     0.066527             0.997142             1.068789   
epss                    0.097353             1.002859             1.102249   
lvdd                    0.348098             0.608071             1.416372   
wallmotion-score        0.123532             1.016800             1.131487   

                  cmp to         z         p  -log2(p)  
covariate                                               
age                  0.0  1.798265  0.072135  3.793155  
epss                 0.0  2.078379  0.037674  4.730271  
lvdd                 0.0 -0.346221  0.729177  0.455660  
wallmotion-score     0.0  2.571040  0.010139  6.623888

In [21]:
# Assess significance
cph.summary['p']

covariate
age                 0.072135
epss                0.037674
lvdd                0.729177
wallmotion-score    0.010139
Name: p, dtype: float64

In [22]:
prediction = cph.predict_survival_function(ecg_df2)

In [23]:
prediction

0         1         2         3         4         5         6    \
0.0   0.889903  0.901576  0.942905  0.895934  0.838613  0.927358  0.651288   
1.0   0.837645  0.854391  0.914579  0.846283  0.765426  0.891773  0.521379   
2.0   0.820799  0.839110  0.905262  0.830239  0.742317  0.880136  0.483857   
3.0   0.803968  0.823809  0.895857  0.814190  0.719468  0.868425  0.448371   
4.0   0.787298  0.808618  0.886447  0.798276  0.697077  0.856739  0.415132   
5.0   0.761825  0.785336  0.871873  0.773920  0.663327  0.838713  0.367854   
6.0   0.753323  0.777546  0.866956  0.765782  0.652189  0.832650  0.352987   
7.0   0.736458  0.762064  0.857118  0.749619  0.630283  0.820549  0.324794   
9.0   0.736458  0.762064  0.857118  0.749619  0.630283  0.820549  0.324794   
10.0  0.727915  0.754206  0.852092  0.741424  0.619282  0.814382  0.311157   
11.0  0.727915  0.754206  0.852092  0.741424  0.619282  0.814382  0.311157   
12.0  0.718919  0.745922  0.846768  0.732789  0.607771  0.807861  0.297253   
13.0  0.718919  0.745922  0.846768  0.732789  0.607771  0.807861  0.297253   
15.0  0.708943  0.736720  0.840826  0.723205  0.595090  0.800595  0.282368   
16.0  0.708943  0.736720  0.840826  0.723205  0.595090  0.800595  0.282368   
17.0  0.698543  0.727112  0.834586  0.713206  0.581967  0.792982  0.267437   
19.0  0.677920  0.708012  0.822075  0.693353  0.556237  0.777765  0.239540   
20.0  0.667019  0.697891  0.815386  0.682844  0.542796  0.769656  0.225682   
21.0  0.645099  0.677480  0.801768  0.661683  0.516106  0.753206  0.199594   
22.0  0.633908  0.667031  0.794728  0.650864  0.502657  0.744732  0.187158   
23.0  0.633908  0.667031  0.794728  0.650864  0.502657  0.744732  0.187158   
24.0  0.633908  0.667031  0.794728  0.650864  0.502657  0.744732  0.187158   
25.0  0.633908  0.667031  0.794728  0.650864  0.502657  0.744732  0.187158   
26.0  0.633908  0.667031  0.794728  0.650864  0.502657  0.744732  0.187158   
27.0  0.633908  0.667031  0.794728  0.650864  0.502657  0.744732  0.187158   
28.0  0.618495  0.652605  0.784929  0.635944  0.484329  0.732974  0.170966   
29.0  0.618495  0.652605  0.784929  0.635944  0.484329  0.732974  0.170966   
31.0  0.600822  0.636014  0.773543  0.618811  0.463599  0.719363  0.153682   
32.0  0.600822  0.636014  0.773543  0.618811  0.463599  0.719363  0.153682   
33.0  0.600822  0.636014  0.773543  0.618811  0.463599  0.719363  0.153682   
34.0  0.600822  0.636014  0.773543  0.618811  0.463599  0.719363  0.153682   
35.0  0.600822  0.636014  0.773543  0.618811  0.463599  0.719363  0.153682   
36.0  0.573402  0.610164  0.755544  0.592169  0.432047  0.697962  0.129433   
37.0  0.573402  0.610164  0.755544  0.592169  0.432047  0.697962  0.129433   
38.0  0.542039  0.580426  0.734425  0.561606  0.396890  0.673034  0.105254   
40.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
41.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
44.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
45.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
46.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
47.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
48.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
49.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
50.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
52.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
53.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
55.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   
57.0  0.509546  0.549414  0.711896  0.529833  0.361543  0.646665  0.083858   

           7         8         9    ...       123       124       125  \
0.0   0.930026  0.964835  0.914858  ...  0.957148  0.872325  0.948315   
1.0   0.895673  0.947081  0.873581  ...  0

In [24]:
from lifelines.utils import k_fold_cross_validation

In [25]:
scores = k_fold_cross_validation(cph, ecg_df2, duration_col='survival_time_hr', event_col='alive', k = 5)
scores

[-1.078442571387482,
 -1.047699155380924,
 -1.2476333455022834,
 -1.133558308270036,
 -1.0058756450146025]